In [3]:
import pandas as pd

from api.data import (store_database_for_eys_gene,
                      parse_lovd,
                      LOVD_PATH,
                      set_lovd_dtypes,
                      )
from api.data import save_lovd_as_vcf
pd.options.display.max_columns = 0

In [4]:
store_database_for_eys_gene("lovd", override=False)

In [5]:
data = parse_lovd(LOVD_PATH + "/lovd_data.txt")

In [6]:
for i in data:
    print(i)
    display(data[i])

Genes


,id,name,chromosome,chrom_band,imprinting,refseq_genomic,refseq_UD,reference,url_homepage,url_external,allow_download,id_hgnc,id_entrez,id_omim,show_hgmd,show_genecards,show_genetests,show_orphanet,note_index,note_listing,refseq,refseq_url,disclaimer,disclaimer_text,header,header_align,footer,footer_align,created_by,created_date,edited_by,edited_date,updated_by,updated_date
0,EYS,eyes shut homolog (Drosophila),6,q12,unknown,NG_023443.2,UD_132085377375,,http://www.LOVD.nl/EYS,,1,21555,346007,612424,1,1,1,1,"<font color=\""#FF0000\"">This database is one o...",,g,http://databases.lovd.nl/shared/refseq/EYS_NM_...,1,,"<font color=\""#FF0000\"">This database is one o...",-1,,-1,00001,2012-02-13 00:00:00,00006,2023-08-30 13:08:19,00006,2024-05-31 11:42:04


Transcripts


,id,geneid,name,id_mutalyzer,id_ncbi,id_ensembl,id_protein_ncbi,id_protein_ensembl,id_protein_uniprot,remarks,position_c_mrna_start,position_c_mrna_end,position_c_cds_end,position_g_mrna_start,position_g_mrna_end,created_by,created_date,edited_by,edited_date
0,00007329,EYS,transcript variant 1,001,NM_001142800.1,,NP_001136272.1,,,,-538,10051,9435,66417118,64429876,,0000-00-00 00:00:00,,


Diseases


...

,id,allele,effectid,chromosome,position_g_start,position_g_end,type,average_frequency,owned_by,VariantOnGenome/DBID,VariantOnGenome/DNA,VariantOnGenome/Frequency,VariantOnGenome/Reference,VariantOnGenome/Restriction_site,VariantOnGenome/Published_as,VariantOnGenome/Remarks,VariantOnGenome/Genetic_origin,VariantOnGenome/Segregation,VariantOnGenome/dbSNP,VariantOnGenome/VIP,VariantOnGenome/Methylation,VariantOnGenome/ISCN,VariantOnGenome/DNA/hg38,VariantOnGenome/ClinVar,VariantOnGenome/ClinicalClassification,VariantOnGenome/ClinicalClassification/Method
0,0000036426,3,50,6,64498971,64498971,subst,0.000742922,00552,EYS_000007,g.64498971A>G,,,,,,Germline,,,0,,,g.63789078A>G,,VUS,
1,0000059881,3,55,6,65655758,65655758,subst,0.00115297,00229,EYS_000001,g.65655758T>G,"ExAC: 60, 19750, 0, 0.003038",{PMID:Neveling 2012:22334370},,Q770P,,Germline,yes,,0,,,g.64945865T>G,,VUS,
2,0000059883,1,11,6,65336143,65336143,subst,0.224189,00229,EYS_000002,g.65336143G>A,"ExAC: 3936, 19366, 441, 0.2032",{PMID:Neveling 2012:22334370},,p.?,unaffected brother also this variant homozygous,Germline,no,,0,,,g.64626250G>A,,benign,
3,0000059884,1,15,6,65300869,65300869,subst,0.000837928,00229,EYS_000003,g.65300869G>A,"ExAC: 12, 19406, 0, 0.0006184",{PMID:Neveling 2012:22334370},,(P1631S),"predicted benign, disease-related variant in o...",Germline,,,0,,,g.64590976G>A,,benign,
4,0000059885,1,11,6,65016998,65016999,del,0,00229,EYS_000004,g.65016998_65016999del,"ExAC: 9866, 18292, 921, 0.5394",{PMID:Neveling 2012:22334370},,6045-4_6045-3del,predicted benign,Germline,yes,,0,,,g.64307105_64307106del,,benign,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,0000987292,0,70,6,64431111,64431111,subst,0,00006,EYS_000662,g.64431111C>T,,"{PMID:Hitti-Malin 2024:38540785}, {DOI:Hitti-M...",,c.8879G>A (Cys2960Tyr),case unsolved,Germline,,,0,,,g.63721215C>T,,likely pathogenic,ACMG
2556,0000987305,0,70,6,66115167,66115167,del,0,00006,EYS_000933,g.66115167del,,"{PMID:Hitti-Malin 2024:38540785}, {DOI:Hitti-M...",,,"no variant 2nd chromosome, case unsolved",Germline,,,0,,,g.65405274del,,likely pathogenic,ACMG
2557,0000987318,0,70,6,64431148,64431148,subst,3.95361E-5,00006,EYS_000067,g.64431148A>G,,"{PMID:Hitti-Malin 2024:38540785}, {DOI:Hitti-M...",,,"no variant 2nd chromosome, case unsolved",Germline,,,0,,,g.63721252A>G,,likely pathogenic,ACMG
2558,0000987322,0,90,6,64430626,64430629,del,0,00006,EYS_000045,g.64430626_64430629del,,"{PMID:Hitti-Malin 2024:38540785}, {DOI:Hitti-M...",,,"no variant 2nd chromosome, case unsolved",Germline,,,0,,,g.63720730_63720733del,,pathogenic,ACMG


Variants_On_Transcripts


,id,transcriptid,effectid,position_c_start,position_c_start_intron,position_c_end,position_c_end_intron,VariantOnTranscript/DNA,VariantOnTranscript/RNA,VariantOnTranscript/Protein,VariantOnTranscript/Exon
0,0000036426,00007329,50,7558,0,7558,0,c.7558T>C,r.(?),p.(Phe2520Leu),38
1,0000059881,00007329,55,2309,0,2309,0,c.2309A>C,r.(?),p.(Gln770Pro),15
2,0000059883,00007329,11,3444,-5,3444,-5,c.3444-5C>T,r.(?),p.(=),22i
3,0000059884,00007329,15,4891,0,4891,0,c.4891C>T,r.(?),p.(Pro1631Ser),26
4,0000059885,00007329,11,6079,-4,6079,-3,c.6079-4_6079-3del,r.(?),p.(=),29i
...,...,...,...,...,...,...,...,...,...,...,...
2555,0000987292,00007329,70,8816,0,8816,0,c.8816G>A,r.(?),p.(Cys2939Tyr),
2556,0000987305,00007329,70,957,0,957,0,c.957del,r.(?),p.(Glu319AspfsTer20),
2557,0000987318,00007329,70,8779,0,8779,0,c.8779T>C,r.(?),p.(Cys2927Arg),
2558,0000987322,00007329,90,9299,0,9302,0,c.9299_9302del,r.(?),p.(Thr3100LysfsTer26),


Screenings_To_Variants


,screeningid,variantid
0,0000000126,0000783293
1,0000000211,0000790459
2,0000001640,0000235838
3,0000016557,0000036426
4,0000033164,0000059884
...,...,...
2163,0000452765,0000987322
2164,0000452851,0000987196
2165,0000452858,0000987333
2166,0000452932,0000987277


In [7]:
set_lovd_dtypes(data)
for i in data:
    print(i)
    display(data[i].info())

Genes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               1 non-null      string        
 1   name             1 non-null      string        
 2   chromosome       1 non-null      Int64         
 3   chrom_band       1 non-null      string        
 4   imprinting       1 non-null      string        
 5   refseq_genomic   1 non-null      string        
 6   refseq_UD        1 non-null      string        
 7   reference        1 non-null      string        
 8   url_homepage     1 non-null      string        
 9   url_external     1 non-null      string        
 10  allow_download   1 non-null      bool          
 11  id_hgnc          1 non-null      Int64         
 12  id_entrez        1 non-null      Int64         
 13  id_omim          1 non-null      Int64         
 14  show_hgmd        1 non-null      bool   

None

Transcripts
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     1 non-null      Int64         
 1   geneid                 1 non-null      string        
 2   name                   1 non-null      string        
 3   id_mutalyzer           1 non-null      Int64         
 4   id_ncbi                1 non-null      string        
 5   id_ensembl             1 non-null      string        
 6   id_protein_ncbi        1 non-null      string        
 7   id_protein_ensembl     1 non-null      string        
 8   id_protein_uniprot     1 non-null      string        
 9   remarks                1 non-null      string        
 10  position_c_mrna_start  1 non-null      Int64         
 11  position_c_mrna_end    1 non-null      Int64         
 12  position_c_cds_end     1 non-null      Int64         
 1

None

Diseases
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            15 non-null     Int64         
 1   symbol        15 non-null     string        
 2   name          15 non-null     string        
 3   inheritance   15 non-null     string        
 4   id_omim       4 non-null      Int64         
 5   tissues       15 non-null     string        
 6   features      15 non-null     string        
 7   remarks       15 non-null     string        
 8   created_by    15 non-null     Int64         
 9   created_date  15 non-null     datetime64[ns]
 10  edited_by     11 non-null     Int64         
 11  edited_date   11 non-null     datetime64[ns]
dtypes: Int64(4), datetime64[ns](2), string(6)
memory usage: 1.6 KB


...

None

Variants_On_Transcripts
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2560 entries, 0 to 2559
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           2560 non-null   Int64 
 1   transcriptid                 2560 non-null   Int64 
 2   effectid                     2560 non-null   Int64 
 3   position_c_start             2559 non-null   Int64 
 4   position_c_start_intron      2560 non-null   Int64 
 5   position_c_end               2559 non-null   Int64 
 6   position_c_end_intron        2560 non-null   Int64 
 7   VariantOnTranscript/DNA      2560 non-null   string
 8   VariantOnTranscript/RNA      2560 non-null   string
 9   VariantOnTranscript/Protein  2560 non-null   string
 10  VariantOnTranscript/Exon     2560 non-null   string
dtypes: Int64(7), string(4)
memory usage: 237.6 KB


None

Screenings_To_Variants
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2168 entries, 0 to 2167
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   screeningid  2168 non-null   Int64
 1   variantid    2168 non-null   Int64
dtypes: Int64(2)
memory usage: 38.2 KB


None

In [8]:
save_lovd_as_vcf(data["Variants_On_Genome"], "./lovd.vcf")

KeyError: 'Variants_On_Genome'

In [ ]:
from subprocess import Popen


process = Popen("spliceai -I ./lovd.vcf -O ./lovd_output.vcf -R ../tools/spliceai/hg38.fa -A grch38".split())
process.wait()

In [ ]:
from api.tools import get_revel_scores

chromosome = 6
position = 65655758

results = get_revel_scores(chromosome, position)

display(results)

In [ ]:
from api.data import request_clinvar_api_data

some_id = 1519786
try:
    frame = request_clinvar_api_data(some_id)
    display(frame)
except Exception as e:
    print(e)


In [ ]:
import requests
from api.data import request_clinvar_api_data

gene_id = '1519786'
# with gene_id = '1519787' error is raised

#TODO inside request_clinvar_api_data
# 1. dinamically expand genes to dataframe (might be one, might be more)
# 2. dinamically expand variation_loc to dataframe (might be one, might be more)
frames = request_clinvar_api_data(gene_id)

display(frames)

In [35]:
import requests

gene_id = '1519785'

path = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=clinvar&id={gene_id}&retmode=json"

request = requests.get(path)

if request.status_code != 200:
    raise ValueError(f"Request failed with status code {request.status_code}")

data = request.json()

 # Extract the 'result' part of the JSON
results = data['result']

# Extract the 'uids' part of the JSON
flattened_data = []

for uid in results['uids']:
    entry = results[uid]

    # Using pd.json_normalize to flatten the JSON data
    flattened_entry = pd.json_normalize(entry, sep='_')

    flattened_variation_set = pd.json_normalize(flattened_entry['variation_set'][0], sep='_')
    flattened_variation_xrefs = pd.json_normalize(flattened_variation_set['variation_xrefs'][0], sep='_')
    
    variation_loc_size = len(flattened_variation_set['variation_loc'][0]) 
    for i in range(variation_loc_size):
        flattened_variation_loc = pd.json_normalize(flattened_variation_set['variation_loc'][0][i], sep='_')
        flattened_variation_loc = flattened_variation_loc.add_prefix(f'{i}_')
        flattened_variation_set = pd.concat([flattened_variation_set, flattened_variation_loc], axis=1)
    
    allele_freq_set_size = len(flattened_variation_set['allele_freq_set'][0])
    for i in range(allele_freq_set_size):
        flattened_allele_freq_set = pd.json_normalize(flattened_variation_set['allele_freq_set'][0][i], sep='_')
        flattened_allele_freq_set = flattened_allele_freq_set.add_prefix(f'{i}_')
        flattened_variation_set = pd.concat([flattened_variation_set, flattened_allele_freq_set], axis=1)
    
    gene_size = len(flattened_entry['genes'][0])
    for i in range(gene_size):
        flattened_genes = pd.json_normalize(flattened_entry['genes'][0][i], sep='_')
        flattened_genes = flattened_genes.add_prefix(f'{i}_')
        flattened_entry = pd.concat([flattened_entry, flattened_genes], axis=1)

    gremline_classification_trait_set_size = len(flattened_entry['germline_classification_trait_set'][0])
    for i in range(gremline_classification_trait_set_size):
        flattened_germline_classification_trait_set = pd.json_normalize(flattened_entry['germline_classification_trait_set'][0][i], sep='_')
        flattened_germline_classification_trait_set = flattened_germline_classification_trait_set.add_prefix(f'{i}_')
        
        trait_xrefs_size = len(flattened_germline_classification_trait_set[f'{i}_trait_xrefs'][0])
        for j in range(trait_xrefs_size):
            flattened_trait_xrefs = pd.json_normalize(flattened_germline_classification_trait_set[f'{i}_trait_xrefs'][0][j], sep='_')
            flattened_trait_xrefs = flattened_trait_xrefs.add_prefix(f'{j}_')

            flattened_germline_classification_trait_set = pd.concat([flattened_germline_classification_trait_set, flattened_trait_xrefs], axis=1)
    
        flattened_germline_classification_trait_set = flattened_germline_classification_trait_set.drop(columns=[f'{i}_trait_xrefs'], axis=1)
        flattened_entry = pd.concat([flattened_entry, flattened_germline_classification_trait_set], axis=1)
        
    # dropping extracted nests
    flattened_entry = flattened_entry.drop(columns=['variation_set', 'genes', 'germline_classification_trait_set'],axis=1)
    flattened_variation_set = flattened_variation_set.drop(columns=['variation_xrefs', 'variation_loc', 'allele_freq_set'], axis=1)

    flattened_variation_set = pd.concat([flattened_variation_set, flattened_variation_xrefs], axis=1)
    flattened_variation_set = pd.concat([flattened_variation_set, flattened_allele_freq_set], axis=1)

    flattened_entry = pd.concat([flattened_entry, flattened_variation_set], axis=1)
    flattened_entry = pd.concat([flattened_entry, flattened_germline_classification_trait_set], axis=1)

    # Append the flattened entry to the list
    flattened_data.append(flattened_entry)

    # Concatenate all flattened entries into a single DataFrame
df = pd.concat(flattened_data, ignore_index=True)

display(df)


,uid,obj_type,accession,accession_version,title,record_status,gene_sort,chr_sort,location_sort,variation_set_name,variation_set_id,molecular_consequence_list,protein_change,fda_recognized_database,supporting_submissions_scv,supporting_submissions_rcv,germline_classification_description,germline_classification_last_evaluated,germline_classification_review_status,germline_classification_fda_recognized_database,clinical_impact_classification_description,clinical_impact_classification_last_evaluated,clinical_impact_classification_review_status,clinical_impact_classification_fda_recognized_database,clinical_impact_classification_trait_set,oncogenicity_classification_description,oncogenicity_classification_last_evaluated,oncogenicity_classification_review_status,oncogenicity_classification_fda_recognized_database,oncogenicity_classification_trait_set,0_symbol,0_geneid,0_strand,0_source,0_trait_name,0_db_source,0_db_id,1_db_source,1_db_id,2_db_source,...,1_stop,1_inner_start,1_inner_stop,1_outer_start,1_outer_stop,1_display_start,1_display_stop,1_assembly_acc_ver,1_annotation_release,1_alt,1_ref,0_source,0_value,0_minor_allele,1_source,1_value,1_minor_allele,2_source,2_value,2_minor_allele,3_source,3_value,3_minor_allele,4_source,4_value,4_minor_allele,db_source,db_id,4_source,4_value,4_minor_allele,0_trait_name,0_db_source,0_db_id,1_db_source,1_db_id,2_db_source,2_db_id,3_db_source,3_db_id
0,1519785,single nucleotide variant,VCV001519785,VCV001519785.,NM_012123.4(MTO1):c.1465+4A>T,,MTO1,06,00000000000073482248,,,[intron variant],,,[SCV002308196],[RCV002024803],Uncertain significance,2022/07/06 00:00,"criteria provided, single submitter",,,1/01/01 00:00,,,[],,1/01/01 00:00,,,[],MTO1,25821,+,submitted,Mitochondrial hypertrophic cardiomyopathy with...,Orphanet,314637,MedGen,C4749921,MONDO,...,74191971,,,,,74191971,74191971,GCF_000001405.25,,,,Exome Aggregation Consortium (ExAC),0.00002,,The Genome Aggregation Database (gnomAD),0.00003,,"The Genome Aggregation Database (gnomAD), exomes",0.00004,,Trans-Omics for Precision Medicine (TOPMed),0.00005,,1000 Genomes Project,0.00020,T,dbSNP,555094677,1000 Genomes Project,0.00020,T,Mitochondrial hypertrophic cardiomyopathy with...,Orphanet,314637,MedGen,C4749921,MONDO,MONDO:0013865,OMIM,614702
